In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from torch.optim import Adam
from tqdm import tqdm
from dotenv import load_dotenv

from transformers import RobertaModel, RobertaTokenizer
from torch.utils.data.sampler import WeightedRandomSampler

In [2]:
load_dotenv()

True

In [3]:
wandb.login()

wandb: Currently logged in as: afifaniks. Use `wandb login --relogin` to force relogin


True

In [4]:
def load_dataframe(path: str) -> pd.DataFrame:
    return pd.read_csv(path)

In [5]:
import random

def clean_data(df):
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    df["text"] = df['text'].str.replace(" +", " ", regex=True)

    return df
    
def prepare_dataframe(df: pd.DataFrame, minimum_contribution = 300) -> pd.DataFrame:
    # developers = df["owner"].value_counts()
    # filtered_developers = developers.index[developers >= minimum_contribution]
    # df = df[df["owner"].isin(filtered_developers)]

    df["text"] = df.apply(lambda x: "Title: " + str(x["issue_title"]) + "\nDescription: " + str(x["description"]), axis=1)
    
    min_length = 15
    df = df[df["text"].str.len().gt(min_length)]

    owners = list(set(df["owner"]))
    keep_random = random.sample(owners, 100)
    df = df[df["owner"].isin(keep_random)]

    df["owner_id"] = pd.factorize(df["owner"])[0]

    return df

In [6]:
data_path = "/home/mdafifal.mamun/notebooks/triagerX/notebook/data/deeptriage/classifier_data_20.csv"

df = load_dataframe(data_path)
df = prepare_dataframe(df)
df = clean_data(df)

In [7]:
len(df["owner"].unique())

100

In [8]:
df["owner"].value_counts().plot(kind="pie")

<Axes: ylabel='count'>

In [9]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=22),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

7316 915 915


/home/mdafifal.mamun/miniconda3/envs/triagerx/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [10]:
df["owner_id"].value_counts()

owner_id
22    488
20    373
2     339
25    332
3     332
     ... 
94     21
55     20
44     20
65     20
97     20
Name: count, Length: 100, dtype: int64

In [11]:
counts = np.bincount(df_train["owner_id"])
labels_weights = 1. / counts
weights = labels_weights[df_train["owner_id"]]
sampler = WeightedRandomSampler(weights, len(weights))

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.tokenizer = tokenizer
        self.labels = [label for label in df['owner_id']]
        self.texts = [self.tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [13]:
class Classifier(nn.Module):
    def __init__(self, model_name: str, output_size, embed_size=1024, dropout = 0.1) -> None:
        super().__init__()
        self.base_model = RobertaModel.from_pretrained(model_name, output_hidden_states=True)
        filter_sizes = [3, 4, 5, 6]
        num_filters = 256
        self._tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.convs1 = nn.ModuleList([nn.Conv2d(4, num_filters, (K, embed_size)) for K in filter_sizes])
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, output_size)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        x = self.base_model(input_ids, attention_mask = attention_mask)[2][-4:]
        x = torch.stack(x, dim=1)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  
        x = torch.cat(x, 1)
        x = self.dropout(x)
        logit = self.fc1(x)  

        return self.relu(logit)
    
    def tokenizer(self) -> RobertaTokenizer:
        return self._tokenizer


In [14]:
def train(model, tokenizer, train_data, val_data, learning_rate, epochs, out_file, run, batch_size = 25):
    train, val = Dataset(train_data, tokenizer), Dataset(val_data, tokenizer)

    wandb.init(
        # Set the project where this run will be logged
        project="triager_x", 
        # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        name=f"deeptriage_{run}", 
        # Track hyperparameters and run metadata
        config={
        "learning_rate": learning_rate,
        "architecture": "Roberta-CNN",
        "dataset": "deeptriage",
        "epochs": epochs,
    })
    

    train_dataloader = torch.utils.data.DataLoader(
        train, batch_size=batch_size, sampler=sampler
    )
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    best_loss = float("inf")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
        print("Using CUDA...")
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        print(f"Epoch: {epoch_num}")
        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
            train_label = train_label.to(device)
            mask = train_input["attention_mask"].to(device)
            input_id = train_input["input_ids"].squeeze(1).to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input["attention_mask"].to(device)
                input_id = val_input["input_ids"].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        val_loss = total_loss_val / len(val_data)

        log = f"Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}"
        print(log)

        wandb.log({
            "train_acc": total_acc_train / len(train_data), 
            "train_loss": total_loss_train / len(train_data),
            "val_acc": total_acc_val / len(val_data),
            "val_loss": total_loss_val / len(val_data)
            })

        if val_loss < best_loss:
            print("Found new best model. Saving weights...")
            torch.save(model.state_dict(), out_file)
            best_loss = val_loss
    
    wandb.finish()

EPOCHS = 30
num_classes = len(df["owner_id"].unique())
print(f"Number of classes: {num_classes}")
model = Classifier("roberta-base", num_classes, 768)
tokenizer = model.tokenizer()
LR = 1e-6

train(model, tokenizer, df_train, df_val, LR, EPOCHS, "deeptriage_weighted_sampler.pt", "weighted_sampler_100_classes")

Number of classes: 100


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using CUDA...
Epoch: 0


100%|██████████| 293/293 [03:01<00:00,  1.62it/s]


Epochs: 1 | Train Loss:  0.185                 | Train Accuracy:  0.019                 | Val Loss:  0.186                 | Val Accuracy:  0.021
Found new best model. Saving weights...
Epoch: 1


...

Epochs: 3 | Train Loss:  0.180                 | Train Accuracy:  0.090                 | Val Loss:  0.184                 | Val Accuracy:  0.057
Found new best model. Saving weights...
Epoch: 3


100%|██████████| 293/293 [02:59<00:00,  1.63it/s]


Epochs: 4 | Train Loss:  0.177                 | Train Accuracy:  0.143                 | Val Loss:  0.182                 | Val Accuracy:  0.082
Found new best model. Saving weights...
Epoch: 4


 12%|█▏        | 35/293 [00:22<02:42,  1.59it/s]


KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.metrics import top_k_accuracy_score

# Load Saved Model

In [ ]:
num_classes = len(df["assignees"].unique())
model = Classifier("roberta-base", num_classes, 768)
model.load_state_dict(torch.load("/home/mdafifal.mamun/notebooks/triagerX/deeptriage_weighted_sampler.pt"))
model = model.cuda()
tokenizer = model.tokenizer()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import random

y_preds = []

for i in range(len(df_test)):
    # rand = random.randint(0, len(df_val))
    dx = df_test.iloc[i]

    data = tokenizer(dx["text"], padding='max_length', max_length = 512, truncation=True)
    ids, mask = data["input_ids"], data["attention_mask"]

    id = torch.tensor([ids]).cuda()
    m = torch.tensor([mask]).cuda()

    g = nn.Softmax(dim=1)

    with torch.no_grad():
        y_pred = g(model(id, m))
        
    y_preds.append(y_pred)

In [ ]:
import numpy as np
from sklearn.metrics import top_k_accuracy_score

In [ ]:
y_true = df_test["owner_id"].to_numpy()

In [ ]:
y_numpy = []

for y in y_preds:
    y_numpy.append(y.cpu().numpy())

In [ ]:
y_preds = np.array(y_numpy)[:,0,:]

In [ ]:
top_k_accuracy_score(y_true, y_preds, k=3)

0.7439516129032258

In [ ]:
top_k_accuracy_score(y_true, y_preds, k=5)

0.8407258064516129